In [1]:
google_document_xlsx = "https://docs.google.com/spreadsheets/d/1riSLCvdlQ8kC5DshtqlRAsVEa7R7N2kD/export?format=xlsx"

In [2]:
!wget {google_document_xlsx} -O /home/jovyan/work/prices_scans.xlsx

--2025-04-17 12:32:02--  https://docs.google.com/spreadsheets/d/1riSLCvdlQ8kC5DshtqlRAsVEa7R7N2kD/export?format=xlsx
Resolving docs.google.com (docs.google.com)... 216.58.211.238, 2a00:1450:4026:808::200e
Connecting to docs.google.com (docs.google.com)|216.58.211.238|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-0k-3k-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/968gltualis8rupfg37nlj30o8/1744893120000/113369956649761615778/*/1riSLCvdlQ8kC5DshtqlRAsVEa7R7N2kD?format=xlsx [following]
--2025-04-17 12:32:03--  https://doc-0k-3k-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/968gltualis8rupfg37nlj30o8/1744893120000/113369956649761615778/*/1riSLCvdlQ8kC5DshtqlRAsVEa7R7N2kD?format=xlsx
Resolving doc-0k-3k-sheets.googleusercontent.com (doc-0k-3k-sheets.googleusercontent.com)... 216.58.211.225, 2a00:1450:4026:808::2001
Connecting to doc-0k-3k-sheets.googleusercontent.com (doc-0k-3k-sheets.googleusercont

In [3]:
import pandas as pd

In [5]:
df = pd.read_excel('/home/jovyan/work/prices_scans.xlsx', header=1)

In [8]:
df.set_index("Номер картинки",  inplace=True)

In [10]:
df.head(4)

,Цена правильная,Цена нейросеть,Ошибка,Строка 1,Строка 2,Строка 3,Строка 4,Строка 5
Номер картинки,,,,,,,,
1,54.99,54.99,0.0,Вода СВЯТОЙ ИСТОЧНИК,1.0л,природная питьевая негаз.ПЭТ,NaN,NaN
2,59.99,59.99,0.0,Вода Святой источник,"0,75л",негазированная спорт,NaN,NaN
3,51.99,51.99,0.0,Вода ШИШКИН ЛЕС,питьевая 1л,негазированная,NaN,NaN
4,24.99,24.99,0.0,Вода КРАСНАЯ ЦЕНА,"1,5л",негазированная,NaN,NaN


In [40]:
import re
from collections import Counter

def generate_words_from_phrases(phrases, len_min=1, do_sort=True):
    word_counter = Counter()

    for phrase in phrases:
        if not isinstance(phrase, str):
            continue  # пропускаем нестроковые значения

        
        words = re.split(r"[ \t\n\r\f\v.,!?;:()\"«»—–\-\/]+", phrase.upper().replace("Ё", "Е"))
        words = list(filter(None, words))  # убираем пустые строки, если они есть
        
        # Оставляем только слова без цифр
        filtered_words = [
            word for word in words
            if len(word) > len_min and not re.search(r"\d", word)
        ]
        word_counter.update(filtered_words)

    if do_sort:
        return sorted(word_counter.items(), key=lambda x: x[1], reverse=True)
    else:
        return word_counter.items()


In [27]:
phrases = pd.concat([df["Строка 1"].dropna(), df["Строка 2"].dropna(), df["Строка 3"].dropna()]).astype(str).unique()


In [41]:
sorted_word_counters = generate_words_from_phrases(phrases)

In [42]:
dict(sorted_word_counters).keys()

dict_keys(['ИЗ', 'СО', 'ДЛЯ', 'ДЕТ', 'ПЭТ', 'ВИНО', 'ПИВО', 'НАПИТОК', 'ВКУСОМ', 'ХЛЕБ', 'МОЛ', 'СМАК', 'ВКУС', 'СТ', 'КРАСНАЯ', 'СЫР', 'ОХЛ', 'ЦЕНА', 'GLOBAL', 'VILLAGE', 'ПЕЧЕНЬЕ', 'СВЕТЛОЕ', 'КАТ', 'ГАЗ', 'НАТ', 'ВОДА', 'ВОДКА', 'ШАМПУНЬ', 'РЕСТОРИЯ', 'ЯБЛОКО', 'САХАР', 'ЗАМ', 'ВК', 'АЛК', 'ОБОГ', 'ПЕЛЬМЕНИ', 'КОФЕ', 'НА', 'ЯБЛОКИ', 'НАЧ', 'СОК', 'ФРУТОНЯНЯ', 'АРАХИС', 'МАСЛО', 'СЛ', 'ПАСТ', 'ТВОРОГ', 'СМЕТАНА', 'ЧАЙ', 'МАНГО', 'HOUSE', 'КАРТ', 'БЕЛОЕ', 'ЧИПСЫ', 'ТОМАТЫ', 'ХРУСТ', 'КРАСНОЕ', 'FISH', 'ЯБЛОК', 'ПИТ', 'ПОДСОЛНЕЧНОЕ', 'АР', 'ЯБЛ', 'НАБОР', 'ВОЛОС', 'КАША', 'МИНИ', 'ИНДЕЙКИ', 'СВ', 'ЧЕР', 'РАСТ', 'COLGATE', 'ГЕЛЬ', 'СЛИВКИ', 'СЛОЙКА', 'БЕЛЫЙ', 'ДСК', 'АЙДИГО', 'СОРТ', 'БАНАН', 'МОЛОЧНАЯ', 'ВОССТ', 'САХ', 'КЛУБ', 'ДОБРЫЙ', 'НЕКТАР', 'КРЕМ', 'ПЮРЕ', 'ФРУТО', 'НЯНЯ', 'HONEY', 'KID', 'РАСТИШКА', 'СОУС', 'ЧЕРНЫЙ', 'КАКАО', 'ЧЕРРИ', 'КАРТОФЕЛЬ', 'ПИРОЖНОЕ', 'ХЛЕБЦЫ', 'СВЕТ', 'ЖАР', 'СМЕТАНЫ', 'БАН', 'СЛИВ', 'ПО', 'НАЧИНКОЙ', 'БАЛТИКА', 'ПОЛУСЛАДКОЕ', 'ЯБЛОЧНЫЙ', 'ЗУБ', 'ЩЕТКА'

In [49]:
new_df = pd.DataFrame(sorted_word_counters, columns=["word", "count"])


In [50]:
# Сохраняем в Excel
del(new_df['count'])

In [51]:
new_df.drop_duplicates(subset=['word'], inplace=True)
new_df.reset_index(drop=True, inplace=True)
new_df.to_csv('/home/jovyan/work/words.txt', index=False, encoding='utf-8', header=False)